In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna
import os

In [2]:
predicciones_path = 'C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/'

In [4]:
predicciones_v07 = pd.read_csv(predicciones_path + 'predicciones_modelos_v007.csv')
predicciones_v08 = pd.read_csv(predicciones_path + 'predicciones_modelos_v008.csv')

In [5]:
predicciones = pd.merge(predicciones_v07, predicciones_v08, on='numero_de_cliente', how='inner')

In [6]:
predicciones['promedio'] = predicciones.drop(columns=['numero_de_cliente']).mean(axis=1)

In [7]:
# ordeno de mayor probabilidad de baja a menor
idx = np.argsort(predicciones['promedio'])[::-1]
predicciones = predicciones.iloc[idx]

In [8]:
predicciones.head()

,numero_de_cliente,pred_lgm_v007_1149,pred_lgm_v007_4836,pred_lgm_v007_9443,pred_lgm_v007_7483,pred_lgm_v007_3492,pred_lgm_v007_6287,pred_lgm_v007_830,pred_lgm_v007_3294,pred_lgm_v007_5932,...,pred_lgm_v008_4836,pred_lgm_v008_9443,pred_lgm_v008_7483,pred_lgm_v008_3492,pred_lgm_v008_6287,pred_lgm_v008_830,pred_lgm_v008_3294,pred_lgm_v008_5932,pred_lgm_v008_13,promedio
43168,677914828,0.999929,0.999932,0.999880,0.999886,0.999782,0.999755,0.999890,0.999944,0.999863,...,0.999992,0.999997,0.999998,0.999994,0.999992,0.999997,0.999991,0.999993,0.999986,0.999935
44849,954734591,0.999878,0.999887,0.999877,0.999913,0.999785,0.999919,0.999949,0.999946,0.999807,...,0.999989,0.999942,0.999978,0.999969,0.999989,0.999985,0.999958,0.999965,0.999989,0.999927
85512,665828559,0.999829,0.999746,0.999749,0.999807,0.999828,0.999851,0.999798,0.999781,0.999845,...,0.999993,0.999981,0.999996,0.999998,0.999995,0.999995,0.999994,0.999993,0.999997,0.999895
143140,1390509893,0.999914,0.999867,0.999907,0.999746,0.999877,0.999901,0.999894,0.999869,0.999939,...,0.999890,0.999865,0.999836,0.999884,0.999922,0.999912,0.999826,0.999937,0.999951,0.999892
92127,467414436,0.999769,0.999796,0.999785,0.999808,0.999787,0.999812,0.999783,0.999700,0.999870,...,0.999976,0.999985,0.999994,0.999997,0.999989,0.999993,0.999993,0.999995,0.999993,0.999891


In [9]:
puntos_corte = np.linspace(10000, 12000, 25).astype(int)
puntos_corte

array([10000, 10083, 10166, 10250, 10333, 10416, 10500, 10583, 10666,
       10750, 10833, 10916, 11000, 11083, 11166, 11250, 11333, 11416,
       11500, 11583, 11666, 11750, 11833, 11916, 12000])

In [10]:
counter = 0

for corte in puntos_corte:
    
    envios = np.zeros(len(predicciones), dtype=int)
    envios[:corte] = 1
    predicciones['Predicted'] = envios
    
    output = predicciones[['numero_de_cliente', 'Predicted']]
        
    counter += 1
    
    file_name = f'results_v010_{counter}.csv'
    output_path = predicciones_path + file_name
    
    output.to_csv(output_path, index=False)
            
    print(f'{counter}. corte: {corte}, {output_path}')

1. corte: 10000, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_1.csv
2. corte: 10083, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_2.csv
3. corte: 10166, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_3.csv
4. corte: 10250, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_4.csv
5. corte: 10333, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_5.csv
6. corte: 10416, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_6.csv
7. corte: 10500, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_7.csv
8. corte: 10583, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_8.csv
9. corte: 10666, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_9.csv
10. corte: 10750, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_10.csv
11. corte: 10833, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v010/results_v010_11.csv
12. corte: 10916, C:/Eugenio/Maestria/DMEyF/exp/competencia_3